<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#import-and-paths" data-toc-modified-id="import-and-paths-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>import and paths</a></span></li><li><span><a href="#training" data-toc-modified-id="training-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>training</a></span><ul class="toc-item"><li><span><a href="#start-probability" data-toc-modified-id="start-probability-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>start probability</a></span></li><li><span><a href="#emission-probability" data-toc-modified-id="emission-probability-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>emission probability</a></span></li><li><span><a href="#transition-probability" data-toc-modified-id="transition-probability-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>transition probability</a></span></li></ul></li><li><span><a href="#Viterbi--Algorithm" data-toc-modified-id="Viterbi--Algorithm-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Viterbi  Algorithm</a></span><ul class="toc-item"><li><span><a href="#Eisner's-Ice-Cream-sample-test" data-toc-modified-id="Eisner's-Ice-Cream-sample-test-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Eisner's Ice Cream sample test</a></span></li></ul></li><li><span><a href="#train-and-test" data-toc-modified-id="train-and-test-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>train and test</a></span></li><li><span><a href="#learning-curve" data-toc-modified-id="learning-curve-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>learning curve</a></span></li></ul></div>

# import and paths

In [1]:
# python version: 3.7.6
# conllu version: 4.5.3
# Matplotlib version: 3.1.3

# total run time: about 70s

In [2]:
from conllu import parse
from collections import Counter
import matplotlib.pyplot as plt
import random

In [3]:
# all paths
path_train = "de_gsd-ud-train.conllu"
path_test  = "de_gsd-ud-test.conllu"

# training 

## start probability

In [4]:
def start_prob(sentences, pos_tags):
    # all possibilities of first word
    tag = 0
    first_tag = []
    for i in range(0, len(sentences)):
        first_tag.append(sentences[i][0]['upos'])
        
    # count
    first_tag_counter = Counter(first_tag)
    
    # tags that do not appear, give 0
    for tag in pos_tags:
        if tag not in first_tag_counter:
            first_tag_counter[tag] = 0
            
    # From count to probability
    dict_start = {item : (count/ len(sentences)) for item, count in first_tag_counter.items()}
    
    return dict_start  

## emission probability

In [5]:
def emission_prob(sentences, pos_tags):
    # double layer dict storing emission probability
    dict_emission_count = {}
    for tag in pos_tags:
        dict_emission_count[tag] = {}
        
    # all sentences
    for sentence in sentences:
        # all words in a sentence
        for word in sentence:
            # count
            if word['form'] not in dict_emission_count[word['upos']]:
                dict_emission_count[word['upos']][word['form']] = 1
            else:
                dict_emission_count[word['upos']][word['form']] += 1          
                
    # from count to probability 
    dict_emission = {}
    for tag, second_dict in dict_emission_count.items():
        sum_one_dict = sum(second_dict.values())
        dict_emission[tag] = {}

        for sec_tag, count in second_dict.items():
            dict_emission[tag][sec_tag] = ((count+1) / (sum_one_dict + len(dict_emission_count[tag])))

    return dict_emission   

## transition probability

In [6]:
def transition_prob(sentences, pos_tags):
    dict_transition_count = {}
    for tag in pos_tags:
        dict_transition_count[tag] = {}
        
    for sentence in sentences:
        leng = len(sentence)
        for idx in range(leng - 1):
            this_tag = sentence[idx]['upos']
            next_tag = sentence[idx+1]['upos']

            if next_tag not in dict_transition_count[this_tag]:
                dict_transition_count[this_tag][next_tag] = 1
            else:
                dict_transition_count[this_tag][next_tag] += 1

    # make sure every possibility is in the dict
    for previous_tag in pos_tags:
        for tag in pos_tags:
            if tag not in dict_transition_count[previous_tag]:
                dict_transition_count[previous_tag][tag] = 0 
            dict_transition_count[previous_tag][tag] += 1
            
    # from count to probability
    dict_transition = {}

    for tag, second_layer in dict_transition_count.items():
        sum_one_dict = sum(second_layer.values())
        dict_transition[tag] = {}
        for sec_tag, count in second_layer.items():
            dict_transition[tag][sec_tag] = count / sum_one_dict
            
    return dict_transition 

# Viterbi  Algorithm

In [7]:
def Viterbi(sentence_word_list:list, dict_emission:dict, dict_start:dict, dict_transition:dict, pos_tags:list):
    if len(sentence_word_list) < 1:
        return None
    # track best previous tag
    back_pointer_list = []
    # track probabilities
    probs_list = []

    # for the first word
    fir_word = sentence_word_list[0]  
    probs = {}
    # whether word appears in training data
    exist = False

    for tag in pos_tags:
        # make sure tag is in the dict_emission        
        if tag not in dict_emission: 
            dict_emission[tag] = {}
        
        # check if the word exist, if not, 0
        if fir_word not in dict_emission[tag]:
            probs[tag] = 0
        else:
            exist = True
            probs[tag] = dict_start.get(tag, 0) * dict_emission[tag].get(fir_word, 0)
            
    # not exist, use start probability
    if not exist:  
        for tag in pos_tags:
            probs[tag] = dict_start.get(tag, 0)

    probs_list.append(probs)

    # forward step
    for i in range(1, len(sentence_word_list)):
        probs = {}
        back_pointer = {}
        # probabilities of last word
        last_prob = probs_list[-1]
        word = sentence_word_list[i]

        # whether word appears in the training data
        exist2 = False
        for tag in pos_tags:
            if word not in dict_emission[tag]:
                # if not appears in emission dict, 0
                dict_emission[tag][word] = 0
            else:
                exist2 = True

        if not exist2:
            # if not appears in training data, 1
            for tag in pos_tags:
                dict_emission[tag][word] = 1

        for tag in pos_tags:
            # largest probability for present tag
            (probs[tag], back_pointer[tag]) = max(
                (last_prob[previous_tag] * dict_transition[previous_tag].get(tag, 0) * dict_emission[tag].get(word, 0), previous_tag)
                for previous_tag in pos_tags)

        probs_list.append(probs)
        back_pointer_list.append(back_pointer)

    # backward
    best_path = []
    max_likelihood, last_state = max((probs_list[-1][tag], tag) for tag in pos_tags)
    best_path.append(last_state)

    for i in range(len(sentence_word_list) - 2, -1, -1):
        last_state = back_pointer_list[i][last_state]
        best_path.append(last_state)

    best_path.reverse()
    return max_likelihood, best_path

## Eisner's Ice Cream sample test

In [8]:
sample_sentence_word_list = ["3", "1", "3"]
sample_pos_tags = ["H", "C"]

# start
sample_dict_start = {"H": 0.8, "C": 0.2}
# emission
sample_dict_emission = {
    "H": {"1": 0.2, "2": 0.4, "3": 0.4},
    "C": {"1": 0.5, "2": 0.4, "3": 0.1}
}
# transition
sample_dict_transition = {
    "H": {"H": 0.7, "C": 0.3},
    "C": {"H": 0.4, "C": 0.6}
}

sample_max_likelihood, sample_best_path = Viterbi(sample_sentence_word_list, sample_dict_emission, sample_dict_start, sample_dict_transition, sample_pos_tags)
print("Sample Max Likelihood:", sample_max_likelihood)
print("Sample Best Path:", sample_best_path)


Sample Max Likelihood: 0.012544000000000003
Sample Best Path: ['H', 'H', 'H']


Max Likelihood: 0.012544000000000003  

Best Path: ['H', 'H', 'H']

# train and test

In [9]:
# reader
def read_data(path):
    with open(path,'r', encoding='utf-8') as file:
        data = file.read()
    sentences = parse(data)
    return sentences 

In [10]:
# train dataset
train_dataset = read_data(path_train)

In [11]:
# test dataset
test_dataset = read_data(path_test)

In [12]:
# get tags by collecting all tags
tag_count_set = []
for sentence in test_dataset:
    for word_info in sentence:
        tag_count_set.append(word_info['upos'])
        
# list->set filter and back to list (all tags)
pos_tags = list(set(tag_count_set))

In [13]:
# get probabilities of emission, start and transition
dict_emission = emission_prob(train_dataset, pos_tags)
dict_start = start_prob(train_dataset, pos_tags)
dict_transition = transition_prob(train_dataset, pos_tags)

In [14]:
# prediction and accuracy
all_tags_num = 0
all_right_tags = 0

for i in range(len(test_dataset)):
    sentence_test = []
    tag_test = []
    for word in test_dataset[i]:
        sentence_test.append(word['form'])  # form of words
        tag_test.append(word['upos'])      # real tags

    # Viterbi
    x, k = Viterbi(sentence_test, dict_emission, dict_start, dict_transition, pos_tags)
    
    # compare predicted tags: k  & real tags: tag_test
    for j in range(len(k)):
        if k[j] == tag_test[j]:
            all_right_tags += 1
        all_tags_num += 1

# accuracy
print(f"Accuracy:{all_right_tags / all_tags_num:.2%}")


Accuracy:90.45%


# learning curve

In [15]:
# learning curve with increasing numbers of training data(total number of about 14000 sentences)
def learning_curve(train_dataset, test_dataset, pos_tags:list, step:int):
    # step: after how many sentences we process, we draw a point
    
    # shuffle
    random.shuffle(train_dataset)
    # list of x coordinates and y coordinates
    training_sizes = []
    accuracies = []

    # training size increases
    for size in range(step, len(train_dataset) + 1, step):
        # first group of training data; data increase gradually 
        incremental_train_data = train_dataset[:size]

        # update probabilities
        dict_emission = emission_prob(incremental_train_data, pos_tags)
        dict_start = start_prob(incremental_train_data, pos_tags)
        dict_transition = transition_prob(incremental_train_data, pos_tags)

        # evaluated on test dataset
        total_tags = 0
        correct_tags = 0
        
        # predict test data tags
        for i in range(len(test_dataset)):
            sentence_test = [word['form'] for word in test_dataset[i]]
            tag_test = [word['upos'] for word in test_dataset[i]]

            # predict tags
            max_lklihood, predicted_tags = Viterbi(sentence_test, dict_emission, dict_start, dict_transition, pos_tags)
            # compare prediction tag and real tags & count
            for j in range(len(predicted_tags)):
                if predicted_tags[j] == tag_test[j]:
                    correct_tags += 1
                total_tags += 1

        # calculate accuracies
        accuracy = correct_tags / total_tags
        # sizes list as x coordinates
        training_sizes.append(size)
        # accuracy list as y coordinates
        accuracies.append(accuracy)

    # draw learning curve
    plt.plot(training_sizes, accuracies)
    plt.title('Learning Curve')
    plt.xlabel('Number of training data used')
    plt.ylabel('Accuracy')
    plt.show()

In [ ]:
# running time: around 1 minute
learning_curve(train_dataset, test_dataset, pos_tags, step = 500)

Analysis: as the number of training data increases, accuracy increases. Finally, accuracy reaches about 90.5%.